In [1]:
import numpy as np 

In [2]:
M = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))

def calc_neighs(field, i, j):
    """ Calculate number of neighbors alive (assuming square field) """
    neighs = 0
    n = len(field)
    for m in M:
        row_idx = m[0] + i
        col_idx = m[1] + j
        if 0 <= row_idx < n and 0 <= col_idx < n:
            if field[row_idx][col_idx]:
                neighs += 1
    return neighs

def make_move(field, moves=1):
    """ Make a move forward according to Game of Life rules """
    n = len(field)
    cur_field = field
    for _ in range(moves):
        new_field = np.zeros((n, n), dtype='uint8')
        for i in range(n):
            for j in range(n):
                neighs = calc_neighs(cur_field, i, j)
                if cur_field[i][j] and neighs == 2:
                    new_field[i][j] = 1
                if neighs == 3:
                    new_field[i][j] = 1
        cur_field = new_field
    return cur_field

def generate_population(size, random_state=-1):
    """
    Generating initial population of individual solutions
    :return: initial population as a list of 20x20 arrays
    """
    if random_state != -1:
        np.random.seed(random_state)
    initial_states = np.split(np.random.binomial(1, 0.5, (20 * size, 20)).astype('uint8'), size)
    return [make_move(state, 5) for state in initial_states]



def generate_problem():
    """ Generates example problem """
    np.random.seed(42)
    board = np.random.binomial(1, 0.01, (20, 20)).astype('uint8')
    # toad
    board[2, 3:6] = 1
    board[3, 2:5] = 1
    # glider 1
    board[9, 5:8] = 1
    board[8, 7] = 1
    board[7, 6] = 1
    # glider 2
    board[16, 3:6] = 1
    board[15, 5] = 1
    board[14, 4] = 1
    # beacon 1
    board[2:4, 14:16]=1
    board[4:6, 16:18]=1
    # beacon 2
    board[14:16, 14:16]=1
    board[16:18, 16:18]=1
    return board, make_move(board, 3)

def fitness(start_field, end_field, delta):
    """
    Calculate fitness for particular candidate (start configuration of the field)
    :param start_field: candidate (start configuration)
    :param end_field: target (stop configuration)
    :param delta: number of steps to proceed before comparing to stop configuration
    :return: value in range [0, 1] that indicates fractions of cells that match their state
    """
    candidate = make_move(start_field, moves=delta)
    return (candidate == end_field).sum() / 400

      
def score_population(population, target, delta):
    """
    Apply fitness function for each gene in a population
    :param population: list of candidate solutions
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :return: list of scores for each solution
    """
    return [fitness(gene, target, delta) for gene in population]


def selection(population, scores, retain_frac=0.8, retain_random=0.05):
    """
    Apply selection operator to the population
    :param population: list of candidate solutions
    :param scores: list of score associated with each individual
    :param retain_frac: percent of top individuals to retain
    :param retain_random: chance of retaining sub-optimal individuals in the population
    """
    retain_len = int(len(scores) * retain_frac)
    sorted_indices = np.argsort(scores)[::-1]
    population = [population[idx] for idx in sorted_indices]
    selected = population[:retain_len]
    leftovers = population[retain_len:]

    for gene in leftovers:
        if np.random.rand() < retain_random:
            selected.append(gene)
    return selected

def mutate(field, switch_frac=0.1):
    """ Inplace mutation of the provided field """
    a = np.random.binomial(1, switch_frac, size=(20, 20)).astype('bool')
    field[a] += 1
    field[a] %= 2
    return field


def crossover(mom, dad):
    """ Take two parents, return two children, interchanging half of the allels of each parent randomly """
    select_mask = np.random.binomial(1, 0.5, size=(20, 20)).astype('bool')
    child1, child2 = np.copy(mom), np.copy(dad)
    child1[select_mask] = dad[select_mask]
    child2[select_mask] = mom[select_mask]
    return child1, child2


def evolve(population, target, delta, retain_frac=0.8, retain_random=0.05, mutate_chance=0.05):
    """
    Evolution step
    :param population: list or candidate solutions
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :param retain_frac: percent of top individuals to proceed into the next genetation
    :param retain_random: chance of retaining sub-optimal individual
    :param mutate_chance: chance of mutating the particular individual
    :return: new generation of the same size
    """
    scores = score_population(population, target, delta)
    next_population = selection(population, scores, retain_frac=retain_frac, retain_random=retain_random)
    
    # mutate everyone expecting for the best candidate
    for gene in next_population[1:]:
        if np.random.rand() < mutate_chance:
            mutate(gene)

    places_left = len(population) - len(next_population)
    children = []
    parent_max_idx = len(next_population) - 1
    while len(children) < places_left:
        mom_idx, dad_idx = np.random.randint(0, parent_max_idx, 2)
        if mom_idx != dad_idx:
            child1, child2 = crossover(next_population[mom_idx], next_population[dad_idx])
            children.append(child1)
            if len(children) < places_left:
                children.append(child2)
    next_population.extend(children)
    return next_population



def solve(target, delta, population_size=200, n_generations=300):
    """
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :param n_generations: number of evolution generations. Overrides initialization value if specified
    :return: 20x20 array that represents the best start field found and associated fitness value
    """
    population = generate_population(population_size)
    for generation in range(n_generations):
        population = evolve(population, target, delta)
        if generation == 0:
            print("Generation #: best score")
        elif generation % 50 == 0:
            print("Generation ", generation, ": ", fitness(population[0], target, delta))
    return population[0]

# Implementación del Game of Life

In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

# Initialize the grid with random alive (1) and dead (0) cells
def initialize_grid(size):
    return np.random.choice([0, 1], size*size, p=[0.8, 0.2]).reshape(size, size)

# Define the function to update the grid based on the Game of Life rules
def update_grid(grid):
    new_grid = grid.copy()
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            # Calculate the number of live neighbors
            live_neighbors = np.sum(grid[max(0, i-1):min(i+2, grid.shape[0]), max(0, j-1):min(j+2, grid.shape[1])]) - grid[i, j]
            
            # Apply the Game of Life rules
            if grid[i, j] == 1:
                if live_neighbors < 2 or live_neighbors > 3:
                    new_grid[i, j] = 0
            else:
                if live_neighbors == 3:
                    new_grid[i, j] = 1
    return new_grid

# Initialize the grid
size = 20  # Grid size
grid = initialize_grid(size)

# Create output directory for frames
output_dir = 'game_of_life_frames/10steps'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Plot and save each state
def save_grid(grid, step):
    plt.imshow(grid, interpolation='nearest', cmap='binary')
    plt.title(f"Step {step}")
    plt.savefig(f"{output_dir}/frame_{step:03d}.png")
    plt.close()

# Number of steps to simulate
steps = 10

# Save the initial state
save_grid(grid, 0)

# Simulate and save each step
for step in range(1, steps + 1):
    grid = update_grid(grid)
    save_grid(grid, step)

print(f"Saved {steps + 1} frames in {output_dir}")

Saved 11 frames in game_of_life_frames/10steps


In [8]:
import cv2
import os

def create_video_from_frames(output_dir, video_name, frame_rate):
    # Get list of frame files sorted by name
    frame_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])
    
    # Read the first frame to get dimensions
    first_frame = cv2.imread(os.path.join(output_dir, frame_files[0]))
    height, width, layers = first_frame.shape

    # Define the codec and create VideoWriter object
    video_path = os.path.join(output_dir, video_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs like 'XVID'
    video = cv2.VideoWriter(video_path, fourcc, frame_rate, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(output_dir, frame_file))
        video.write(frame)

    video.release()
    print(f"Video saved at {video_path}")

# Example usage
output_dir = 'game_of_life_frames/10steps'
video_name = 'game_of_life.mp4'
frame_rate = 10  # frames per second

create_video_from_frames(output_dir, video_name, frame_rate)

Video saved at game_of_life_frames/10steps/game_of_life.mp4


Leer el archivo de trian y generar los tableros de train y test

In [12]:
train_csv = pd.read_csv('/Users/victorgutierrezgarcia/Desktop/MUTSC/2º Cuatri /BILN/biln_project/bioinspired_learning/data/train.csv')
train_csv.head()

,id,delta,start.1,start.2,start.3,start.4,start.5,start.6,start.7,start.8,...,stop.391,stop.392,stop.393,stop.394,stop.395,stop.396,stop.397,stop.398,stop.399,stop.400
0,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,5,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,2,0,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
